In [1]:
import numpy as np
import pandas as pd
import os
import SimpleITK as sitk
import subprocess
from utils import get_landmark,register,transform,extract,process_fcsv_to_txt, extract_all_objects_mask, pad_and_save_images, add_rows_to_file,extract_output_points ,convert_to_fcsv_with_labels,extract_3d_coordinates, _compute_TRE

# Initial Validation

In [2]:
fixed_landmark=get_landmark(patient_num="08")

for patient_num in range(9,21):
    if patient_num == 9:
        patient_num = f"0{patient_num}"
        moved_landmark = get_landmark(patient_num=patient_num)
        TRE = _compute_TRE(fixed_landmark, moved_landmark)
        TRE = round(TRE, 3)  # Restrict TRE to 3 decimal places
        TRE = np.round(TRE, 3).tolist()  # Restrict TRE_mm to 3 decimal places
        print(f'Patient {patient_num} TRE mean: {TRE} mm')
    else:
        moved_landmark = get_landmark(patient_num=patient_num)
        TRE = _compute_TRE(fixed_landmark, moved_landmark)
        TRE = round(TRE, 3)  # Restrict TRE to 3 decimal places
        TRE = np.round(TRE, 3).tolist()  # Restrict TRE_mm to 3 decimal places
        print(f'Patient {patient_num} TRE mean: {TRE} mm')

Patient 09 TRE mean: 0.748 mm
Patient 10 TRE mean: 0.917 mm
Patient 11 TRE mean: 0.889 mm
Patient 12 TRE mean: 0.919 mm
Patient 13 TRE mean: 0.826 mm
Patient 14 TRE mean: 1.101 mm
Patient 15 TRE mean: 0.874 mm
Patient 16 TRE mean: 1.062 mm
Patient 17 TRE mean: 0.933 mm
Patient 18 TRE mean: 0.822 mm
Patient 19 TRE mean: 0.672 mm
Patient 20 TRE mean: 0.746 mm


# Registration

In [9]:
params_1 = f'parameters\par33\Par0033similarity.txt'
params_2 = f'parameters\par33\Par0033bspline.txt'
elastix_exe = r'D:\uni\Erasmus_Master\University_of_Girona\MIRA\Lab2\elastix-5.0.0-win64\elastix.exe'

patient_num = 8
patient_num = f"0{patient_num}"
mask_fixed, image_fixed,output_fixed= register(patient_num)

for patient_num in range(9,21):
    if patient_num == 9:
        patient_num = f"0{patient_num}"
        mask_moved, image_moved,output_moved= register(patient_num)
        subprocess.run([
        elastix_exe,
        '-f', image_fixed,
        '-m', image_moved,
        '-fMask', mask_fixed,
        # '-mMask', moved_seg,
        '-p', params_1,
        '-p', params_2,
        '-out', output_moved
    ])

        print(f"Registration completed for Patient {patient_num}. Outputs are in {output_moved}")
    else:
        mask_moved, image_moved,output_moved= register(patient_num)
        subprocess.run([
        elastix_exe,
        '-f', image_fixed,
        '-m', image_moved,
        '-fMask', mask_fixed,
        # '-mMask', moved_seg,
        '-p', params_1,
        '-p', params_2,
        '-out', output_moved
    ])

        print(f"Registration completed for Patient {patient_num}. Outputs are in {output_moved}")
        

Mask of all objects (excluding label 0) saved to Training_data_2/cop1/data/NG4108_mask.nrrd
Mask of all objects (excluding label 0) saved to Training_data_2/cop1/data/NG4120_mask.nrrd
Registration completed for Patient 20. Outputs are in par/cop20


In [11]:
transformix_exe = r'D:\uni\Erasmus_Master\University_of_Girona\MIRA\Lab2\elastix-5.0.0-win64\transformix.exe'

for patient_num in range(9,21):
    if patient_num == 9:
        patient_num = f"0{patient_num}"
        edited_file_path, output_dir,transform_params= transform(patient_num)
        subprocess.run([
        transformix_exe,
        '-def', edited_file_path,
        '-out', output_dir,
        '-tp', transform_params
    ])

        print(f"Registration completed for Patient {patient_num}. Outputs are in {output_dir}")
    else:
        edited_file_path, output_dir,transform_params= transform(patient_num)
        subprocess.run([
        transformix_exe,
        '-def', edited_file_path,
        '-out', output_dir,
        '-tp', transform_params
    ])

        print(f"Transformation completed for Patient {patient_num}. Outputs are in {output_dir}")
        

Updated file saved as: Training_data_2/cop1/data/NG4108_Fiducial_template_ALL_edited.txt
Transformation completed for Patient 20. Outputs are in par/cop20/step2


In [12]:
for patient_num in range(9,21):
    if patient_num == 9:
        patient_num = f"0{patient_num}"
        target_array_fixed, target_array_moved=extract(patient_num)
        TRE = _compute_TRE(target_array_fixed, target_array_moved)
        TRE = round(TRE, 3)  # Restrict TRE to 3 decimal places
        TRE = np.round(TRE, 3).tolist()  # Restrict TRE_mm to 3 decimal places
        print(f'Patient {patient_num} TRE mean: {TRE} mm')
    else:
        target_array_fixed, target_array_moved=extract(patient_num)
        TRE = _compute_TRE(target_array_fixed, target_array_moved)
        TRE = round(TRE, 3)  # Restrict TRE to 3 decimal places
        TRE = np.round(TRE, 3).tolist()  # Restrict TRE_mm to 3 decimal places
        print(f'Patient {patient_num} TRE mean: {TRE} mm')

Patient 09 TRE mean: 0.379 mm
Patient 10 TRE mean: 0.477 mm
Patient 11 TRE mean: 0.648 mm
Patient 12 TRE mean: 0.468 mm
Patient 13 TRE mean: 0.43 mm
Patient 14 TRE mean: 0.494 mm
Patient 15 TRE mean: 0.388 mm
Patient 16 TRE mean: 0.363 mm
Patient 17 TRE mean: 0.291 mm
Patient 18 TRE mean: 0.298 mm
Patient 19 TRE mean: 0.453 mm
Patient 20 TRE mean: 0.413 mm
